# DeepLabCut ModelZoo

## Local Implementation of DLC Using Pretrained [ModelZoo](http://www.mackenziemathislab.org/dlc-modelzoo) SuperAnimals for Zero-Shot Labeling 

**Notebook Owner:** [insert name] <br>
**Date:** [insert date]

***

### Installation
Make sure you follow https://deeplabcut.github.io/DeepLabCut/docs/installation.html to install latest version of deeplabcut: 
> `git clone https://github.com/DeepLabCut/DeepLabCut.git`

Make sure the version you download includes a package with modelzoo. Additionally, note which GPU you will be running on when you create the conda environment, as the packages will break if you aren't certain: 
> **Intel Chip:** `conda env create -f DEEPLABCUT.yaml` <br>
> **Apple M1/M2:** `conda env create -f DEEPLABCUT_M1.yaml`

In [11]:
%load_ext autoreload
%autoreload 2
import os
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=Warning)

from dlc_utils.dlc_config import select_session, get_trial_video_list, clean_pretrained_project
from dlc_utils.dlc_downsample import check_for_downsample, downsample_videos

import deeplabcut


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Selecting session: 230717_Aragorn
Checking for video files...
  Video directory: /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230717_Aragorn


100%|██████████| 420/420 [00:00<00:00, 22568.80it/s]

Number of video files: 420


,date,session_num,subject,trial_num,block,condition,correct,error,error_type,behavioral_code_markers,...,blink_raster,blink_raster_window,blink_duration_window,pupil_pre_CS,lick_in_window,blink_in_window,lick_duration,blink_duration_sig,blink_duration_offscreen,eye_distance
0,230717,0,Aragorn,1,1,1,1,0,0,"[9, 100, 101, 102, 103, 104, 105, 106, 107, 10...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.420667,"[7517.0, 7517.0, 7516.0, 7516.0, 7515.0, 7515....",1.0,1.0,0.709333,0.408000,0.816000,104.702938
2,230717,0,Aragorn,3,1,1,1,0,0,"[9, 100, 101, 102, 103, 104, 105, 106, 107, 10...",...,"[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.398000,"[7360.0, 7360.0, 7360.0, 7362.0, 7362.0, 7362....",1.0,1.0,0.900000,0.374667,0.630667,79.473832
3,230717,0,Aragorn,4,1,1,1,0,0,"[9, 100, 101, 102, 103, 104, 105, 106, 107, 10...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.005333,"[7268.0, 7268.0, 7268.0, 7268.0, 7268.0, 7268....",1.0,1.0,0.961333,0.001333,0.337333,76.115814
4,230717,0,Aragorn,5,1,1,1,0,0,"[9, 100, 101, 102, 103, 104, 105, 106, 107, 10...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,"[6841.0, 6841.0, 6842.0, 6842.0, 6842.0, 6842....",1.0,1.0,1.000000,0.000000,0.234000,61.408204
5,230717,0,Aragorn,6,1,1,1,0,0,"[9, 100, 101, 102, 103, 104, 105, 106, 107, 10...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.035333,"[6938.0, 6938.0, 6936.0, 6936.0, 6934.0, 6934....",1.0,1.0,0.784000,0.026000,0.711333,72.015905


***
### Set Video Path
`get_trial_video_list` captures all the videos from the specified video directory path. Set your `VIDEO_TYPE` to the appropriate video extension (i.e. .mp4, .mov).

In [ ]:
ROOT = '/Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/'
VIDEO_ROOT = os.path.join(ROOT, 'tasks', 'rhAirpuff', '8. Probabilistic_Reward_Airpuff_Choice', 'videos')
SESSION_ROOT = os.path.join(ROOT, 'data', 'processed', 'processed_Probabilistic_Reward_Airpuff_Choice')

# Select session
DATE = '230717'
MONKEY = 'Aragorn'

session_df, video_path = select_session(SESSION_ROOT, VIDEO_ROOT, DATE, MONKEY, correct_only=True)
video_path_list = get_trial_video_list(session_df, video_path)
session_df.head(5)

***
### Select SuperAnimal

Select the SuperAnimal model you'd like to fit your videos to. 
Options listed below:
> * full_human <br>
> * full_cat <br>
> * full_dog <br>
> * primate_face <br>
> * mouse_pupil_vclose <br>
> * horse_sideview <br>
> * full_macaque <br>
> * superanimal_topviewmouse <br>
> * superanimal_quadruped <br>

In [2]:
import ipywidgets as widgets
from IPython.display import display

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = widgets.Dropdown(
  options=model_options,
	value=model_options[3],
	description="Choose a DLC ModelZoo model!",
	disabled=True
)
print(f'Model selected: {model_selection.value}')

Model selected: primate_face


***
### Create Project
Initialize your project by setting the <DATE> and <ANIMAL(S)> to title your project. `check_for_downsample` will see how large the video files are, and if necessary (i.e. larger than 640x640), downsamples to a size that makes this implementation reasonable (within 30 min for ~500 videos) on a local computer.

If you don't want to resave all your video files to the new project folder, set the `copy_videos` parameter within `deeplabcut.create_pretrained_project` to be `False`.

Initializing the project also checks for the following:
* Checks that CUDA / GPUs are configured properly via tensorflow installation
  * Refer to [tensorflow documentation](https://www.tensorflow.org/install/pip#windows-native) for more info
  * On my system, I had to open up an instance of terminal with the DEEPLABCUT environment from Anaconda Navigator <br> and run `conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0` even after setting up DLC in order to have it running with GPUs
* Frame size (requires downsampling if greater than 640x640)
* Training the initial video(s) on the appropriate DLC model (i.e. full_macaque)

In [3]:
from PIL import Image
import cv2

project_name = f'{DATE}_{MONKEY}'
your_name = 'rahim'
model2use = model_selection.value
# .mp4 or .avi etc.
videotype = os.path.splitext(video_path_list[0])[-1].lstrip('.')

# Check if videos need to be downsampled
downsample_flag = check_for_downsample(video_path_list)

if downsample_flag:
	video_path_list = downsample_videos(video_path_list)

# Create ModelZoo project
config_path, train_config_path = deeplabcut.create_pretrained_project(
    project_name,
    your_name,
    video_path_list[0:1],
    videotype=videotype,
    model=model2use,
    analyzevideo=True,
    createlabeledvideo=True,
    copy_videos=False, # must leave copy_videos=True
)

Checking frame size...
   Pixel width x height: 320x240
No need to downsample videos
Created "/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/videos"
Created "/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/labeled-data"
Created "/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/training-datasets"
Created "/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/dlc-models"
Attempting to create a symbolic link of the video ...
Created the symlink of /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230717_Aragorn/230717_Aragorn_choice_Cam1_1.mp4 to /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/videos/230717_Aragorn_choice_Cam1_1.mp4
/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/

/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/dlc-models/iteration-0/230717_AragornJul17-trainset95shuffle1/train/pose_cfg.yaml
Analyzing video...
Using snapshot-1030000 for model /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/dlc-models/iteration-0/230717_AragornJul17-trainset95shuffle1


2023-07-17 17:53:22.517996: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Analyzing all the videos in the directory...
Starting to analyze %  /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/videos/230717_Aragorn_choice_Cam1_1.mp4
Loading  /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/videos/230717_Aragorn_choice_Cam1_1.mp4
Duration of video [s]:  5.83 , recorded with  30.0 fps!
Overall # of frames:  175  found with (before cropping) frame dimensions:  320 240
Starting to extract posture


100%|██████████| 175/175 [00:03<00:00, 53.34it/s]


Saving results in /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Analyzing all the videos in the directory...
Filtering with median model /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/videos/230717_Aragorn_choice_Cam1_1.mp4
Saving filtered csv poses!
Plotting results...
Analyzing all the videos in the directory...
Starting to process video: /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/videos/230717_Aragorn_choice_Cam1_1.mp4
Loading /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/videos/230717_Arag

100%|██████████| 175/175 [00:00<00:00, 634.33it/s]


Analyzing all the videos in the directory...
Loading  /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/videos/230717_Aragorn_choice_Cam1_1.mp4 and data.
Plots created! Please check the directory "plot-poses" within the video directory


In [4]:
# Delete the first video in the config file to rerun with new config
clean_pretrained_project(config_path)

Deleting files for rerun...
  Deleted: 230717_Aragorn_choice_Cam1_1DLC_resnet50_230717_AragornJul17shuffle1_1030000_meta.pickle
  Deleted: 230717_Aragorn_choice_Cam1_1DLC_resnet50_230717_AragornJul17shuffle1_1030000_filtered.h5
  Deleted: 230717_Aragorn_choice_Cam1_1DLC_resnet50_230717_AragornJul17shuffle1_1030000.csv
  Deleted: 230717_Aragorn_choice_Cam1_1DLC_resnet50_230717_AragornJul17shuffle1_1030000.h5
  Deleted: 230717_Aragorn_choice_Cam1_1DLC_resnet50_230717_AragornJul17shuffle1_1030000_filtered_labeled.mp4
  Deleted: 230717_Aragorn_choice_Cam1_1DLC_resnet50_230717_AragornJul17shuffle1_1030000_filtered.csv
Done deleting file.


***
### Update Config Parameters
Not a necessary step, but helpful to personalize your videos/analyses to your needs!

> * **dotsize:** size of dots for labeled videos <br>
> * **pcutoff:** threshold for accepting a landmark <br>
> * **skeleton:** depending on the model you are using, you can create a skeleton that connects landmark points together. Below is specific to primate face, but take a look through the config.yaml file generated in your project folder

In [5]:
# Updating the configs within the config.yaml file
edits = {
    'dotsize': 3,  # size of the dots!
    'colormap': 'rainbow',  # any matplotlib colormap
    'alphavalue': 0.2, # transparency of labels
    'pcutoff': 0.5,  # the higher the more conservative the plotting!
    'skeleton': 
         # Right Eye
        [['RightEye_Top', 'RightEye_Inner'], 
         ['RightEye_Inner', 'RightEye_Bottom'],
         ['RightEye_Outer', 'RightEye_Bottom'],
         ['RightEye_Top', 'RightEye_Outer'], 
         # Left Eye
         ['LeftEye_Top', 'LeftEye_Inner'],
         ['LeftEye_Inner', 'LeftEye_Bottom'],
         ['LeftEye_Outer', 'LeftEye_Bottom'],
         ['LeftEye_Top', 'LeftEye_Outer'],
        #  # Top of Head Counter-Clockwise to Lip
        #  ['HeadTop_Mid', 'OutlineRight_Mouth'],
        #  ['OutlineRight_Mouth', 'RightNostrils_Bottom'],
        #  ['RightNostrils_Bottom', 'UpperLip_Centre'],
        #  # Lip Counter-Clockwise to Top of Head
        #  ['UpperLip_Centre', 'OutlineLeft_Mouth'],
        #  ['OutlineLeft_Mouth', 'LeftNostrils_Bottom'],
        #  ['LeftNostrils_Bottom', 'HeadTop_Mid'],
        ],
    'skeleton_color': 'white'
}
deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

{'Task': '230717_Aragorn', 'scorer': 'rahim', 'date': 'Jul17', 'multianimalproject': False, 'identity': None, 'project_path': '/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17', 'video_sets': {'/Users/rahimhashim/Library/CloudStorage/GoogleDrive-rh2898@columbia.edu/.shortcut-targets-by-id/1weRx7ojG3amil91WgRMeTUVxse__Rsyt/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230717_Aragorn/230717_Aragorn_choice_Cam1_1.mp4': {'crop': '0, 320, 0, 240'}}, 'bodyparts': ['HeadTop_Mid', 'RightEarTop_Join', 'RightEarTop_High', 'RightEar_Outer', 'RightEarBottom_Low', 'RightEarBottom_Join', 'RightEar_Tragus', 'OutlineTop_Mid', 'OutlineTop_Right', 'OutlineRight_Brow', 'OutlineRight_Indent', 'OutlineRight_Cheek', 'OutlineRight_Mouth', 'OutlineChin_Mid', 'OutlineLeft_Mouth', 'OutlineLeft_Cheek', 'OutlineLeft_Indent', 'OutlineLeft_Brow', 'OutlineTop_Left', 'LeftEarTop_Join', 'LeftEarTop_High', 'LeftEar_Outer', 'LeftEarBottom_Low', 'LeftEarBottom_Join', 'LeftE

***
### Run Analyses

The key part of the pipeline, where `deeplabcut.analyze_videos` identifies the landmarks for each frame in the included videos. This will take **~30 minutes for ~500 videos** on a Mac M1, so if you want to specify a subset of videos just to make sure it works, set `START_VIDEO` and `END_VIDEO` values. If not, `video_list_subset` can be set to the directory's entire video list `video_path_list[:]`

In [7]:
video_list_subset = video_path_list[:]
# Adding new videos to the config.yaml file
deeplabcut.add_new_videos(config_path, video_list_subset, copy_videos=False, 
            coords=None, extract_frames=False
)

# Analyze specified videos
deeplabcut.analyze_videos(config_path, video_list_subset, 
            videotype, save_as_csv=True
)

# Filter predictions
deeplabcut.filterpredictions(config_path, video_list_subset, videotype=videotype)

# Create labeled videos
deeplabcut.create_labeled_video(
   config_path, video_list_subset, 
   videotype, 
   draw_skeleton=True, 
   filtered=True,
   trailpoints=5,
)

# Plot trajectories
deeplabcut.plot_trajectories(config_path, video_list_subset, videotype, filtered=True)

Attempting to create a symbolic link of the video ...
New videos were added to the project! Use the function 'extract_frames' to select frames for labeling.
Using snapshot-1030000 for model /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230717_Aragorn-rahim-2023-07-17/dlc-models/iteration-0/230717_AragornJul17-trainset95shuffle1
No video(s) were found. Please check your paths and/or 'video_type'.
No video(s) were found. Please check your paths and/or 'videotype'.
No videos found. Make sure you passed a list of videos and that *videotype* is right.


***
### Generate Labeled Videos and Summary Plots
If you'd like to see the labeled videos, run `deeplabcut.create_labeled_video` and it will create labeled videos. Running `deeplabcut.plot_trajectories` creates plots that summarize the distribution of movement for your landmarks.

In [17]:
deeplabcut.plot_trajectories(config_path, video_list_subset, videotype, filtered=True)

Loading  /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230627_Bear/230627_Bear_choice_Cam1_1.mp4 and data.
Loading  /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230627_Bear/230627_Bear_choice_Cam1_4.mp4 and data.
Loading  /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230627_Bear/230627_Bear_choice_Cam1_5.mp4 and data.
Loading  /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230627_Bear/230627_Bear_choice_Cam1_8.mp4 and data.
Loading  /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230627_Bear/230627_Bear_choice_Cam1_10.mp4 and data.
Plots created!